In [74]:
import warnings
warnings.filterwarnings("ignore")

In [75]:
# Import necessary libraries
import os
import joblib
import cv2
import shutil
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import skimage.io
from tqdm import tqdm_notebook as tqdm
import tensorflow as tf
from PIL import Image
#import pytesseract
import argparse
import os
import pandas as pd
from sklearn import preprocessing
import math
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import seaborn as sn
import matplotlib.pyplot as plt
import pickle
from random import shuffle
from keras.optimizers import Adam
import tensorflow as tf
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,precision_score,recall_score
from statistics import mode
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,precision_score,recall_score
from keras.callbacks import ReduceLROnPlateau
import h5py
from keras import models   
from sklearn.metrics import classification_report
from random import randint
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
tf.get_logger().setLevel('ERROR')

In [76]:
data=pd.read_csv('E:/BGB_example_files_test_last/labels/all.csv')
data.head(10)

,E:/BGB_example_files_test_last/images/Purchase_Order/PKM_C454e19051314380-1.png 18
0,E:/BGB_example_files_test_last/images/Internal...
1,E:/BGB_example_files_test_last/images/Labels/P...
2,E:/BGB_example_files_test_last/images/OCB/PKM_...
3,E:/BGB_example_files_test_last/images/OCB/PKM_...
4,E:/BGB_example_files_test_last/images/Delivery...
5,E:/BGB_example_files_test_last/images/Internal...
6,E:/BGB_example_files_test_last/images/Rental_A...
7,E:/BGB_example_files_test_last/images/OCB/PKM_...
8,E:/BGB_example_files_test_last/images/OCB/PKM_...
9,E:/BGB_example_files_test_last/images/Refunds/...


**Utility Methods**

In [77]:
def create_regional_data(path,data_type='train'):
  """
  data_type = train/test/val

  """
  train_dir=path
  for folder in tqdm(os.listdir(train_dir)):
      for image in os.listdir(train_dir+'/'+folder):
          try:
              img = cv2.imread(train_dir+'/'+folder+'/'+image,0)
              height = img.shape[0]
              width = img.shape[1]
              header = img[0:(int(height*0.33)), 0:width]
              footer = img[int(height*0.67):height, 0:width]
              left_body = img[int(height*0.33):int(height*0.67), 0:int(width*0.5)]
              right_body = img[int(height*0.33):int(height*0.67), int(width*0.5):width]

              header_path=data_type+'/header/'+folder+'/'+image.split('/')[-1]
              footer_path=data_type+'/footer/'+folder+'/'+image.split('/')[-1]
              left_path=data_type+'/left_body/'+folder+'/'+image.split('/')[-1]
              right_path=data_type+'/right_body/'+folder+'/'+image.split('/')[-1]
              if(os.path.exists(header_path)==False):
                  cv2.imwrite(header_path, header)
              if(os.path.exists(footer_path)==False):
                  cv2.imwrite(footer_path, footer)
              if(os.path.exists(left_path)==False):
                  cv2.imwrite(left_path, left_body)
              if(os.path.exists(right_path)==False):
                  cv2.imwrite(right_path, right_body)
          except:
            print('Some Error Occured with:',train_dir+'/'+folder+'/'+image)        

In [78]:
def get_data_generator(train_dir,test_dir,img_height, img_width,batch_size):
    """
    method to generate training batches on the go, along with data augmentation
    """
    # Using Data Augmentation for better variance
    train_datagen = ImageDataGenerator(
    rescale = 1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=13,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2)

    train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size, 
    class_mode = "categorical")

    val_datagen = ImageDataGenerator(
    rescale = 1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=13,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2)

    val_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size, 
    class_mode = "categorical")
    return train_generator,val_generator

In [79]:
def plot_confusion_matrix(test_y, predict_y,classes):
     
    C = confusion_matrix(test_y, predict_y)
    # C = 9,9 matrix, each cell (i,j) represents number of points of class i are predicted class j
    """
    This function will plot the consusion matrix.
    """
    A =(((C.T)/(C.sum(axis=1))).T)
   
    B =(C/C.sum(axis=0))
    labels = classes
    cmap=sns.light_palette("green")
    # representing A in heatmap format
    print("-"*50, "Confusion matrix", "-"*50)
    plt.figure(figsize=(17,5))
    sns.heatmap(C, annot=True, cmap=cmap, fmt="0.0f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()

    print("-"*50, "Precision matrix", "-"*50)
    plt.figure(figsize=(17,5))
    sns.heatmap(B, annot=True, cmap=cmap, fmt="0.2f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()
    print("Sum of columns in precision matrix",B.sum(axis=0))
   
    # representing B in heatmap format
    print("-"*50, "Recall matrix"    , "-"*50)
    plt.figure(figsize=(17,5))
    sns.heatmap(A, annot=True, cmap=cmap, fmt="0.2f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()
    print("Sum of rows in precision matrix",A.sum(axis=1))

In [80]:
def eval_model(model,test_generator,limit):
    """
   This function predicts the result of a candidate model
   test_generator : genertaor to evaluate
   model : candidate model
   limit: No of test samples to consider
    
    """
    x,y=test_generator.next()
    y_pred=[]
    y_true=[]
    count=0
    for i in range(len(x)):
        pred=model.predict(np.expand_dims(x[i], axis=0))
        predicted_y=np.argmax(pred)
        y_pred.append(predicted_y)
        y_true.append(np.argmax(y[i]))
        count+=1
        # limit the number of samples to be considered
        if(count==limit):
            break
    return y_pred,y_true

In [81]:
def get_f1_precision_recall(y_pred,y_true):
    """
    This function evaluate the performance(accuracy,f1-score,precision,recall,confusion matrix of the network).
    """
    f1=np.round(f1_score(y_pred,y_true,average='macro'),3)
    precision=np.round(precision_score(y_pred,y_true,average='macro'),3)
    recall=np.round(recall_score(y_pred,y_true,average='macro'),3)
    return f1,precision,recall   

In [82]:
def get_result(model,test_generator):
    res_test=model.evaluate_generator(test_generator)
    print('Test Accuracy:',res_test[1]*100,'%')
    y_pred,y_true=eval_model(model,test_generator,2000)
    f1,precision,recall=get_f1_precision_recall(y_pred,y_true)
    print('Macro F1 Score:',f1)
    print('Precision Score:',precision)
    print('Recall Score:',recall)

In [83]:
# Method to check if all the classes are present in test data
def sanity_check(y_val):
  labels=[]
  for i in range(len(y_val)):
      labels.append(np.argmax(y_val[i]))
  print(set(labels))

In [84]:
# Internal method to calculate performance metrics
def get_result_(y_pred,y_true,classes):
    print('-----Model Evaluation------')
    accuracy=np.round(accuracy_score(y_pred,y_true),2)
    print('Accuracy Score:',accuracy)
    f1,precision,recall=get_f1_precision_recall(y_pred,y_true)
    print('Macro F1 Score:',f1)
    print('Precision Score:',precision)
    print('Recall Score:',recall)
    plot_confusion_matrix(y_true,y_pred,classes)

In [85]:
# Method to generate test data for final model evaluation
# Note: we are not shuffeling the data here as we need to take the wholistic and regional data for an image in sequence
def generate_test_data(val_path,num_samples,img_height, img_width):
    """ This method will generate data to generate features for meta classifier"""
    datagen = ImageDataGenerator(rescale = 1./255,featurewise_center=True,
    featurewise_std_normalization=True,rotation_range=13,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2)
    val_generator = datagen.flow_from_directory(
    val_path,target_size = (img_height, img_width),batch_size =num_samples , class_mode = "categorical",shuffle=False)
    x_val,y_val=val_generator.next()
    return x_val,y_val

In [86]:
# Method to evaluate final model
def eval_stacked_model(weights,x_val,x_val_top,x_val_bottom,x_val_left,x_val_right,y_val
    ,model_final_holistic,model_final_top,model_final_bottom,model_final_left,model_final_right,classes,matrix=False):
    y_true=[]
    y_pred=[]
    for i in range(len(x_val)):
        pred_whole=model_final_holistic.predict(np.expand_dims(x_val[i], axis=0))
        pred_top=model_final_top.predict(np.expand_dims(x_val_top[i], axis=0))
        pred_bottom=model_final_bottom.predict(np.expand_dims(x_val_bottom[i], axis=0))
        pred_left=model_final_left.predict(np.expand_dims(x_val_left[i], axis=0))
        pred_right= model_final_right.predict(np.expand_dims(x_val_right[i], axis=0))  
        try:
            # we will concatenate the predictions of the candidate models
            predicted_y=np.argmax(pred_whole*weights[0]+pred_top*weights[1]+pred_bottom*weights[2]
                               +pred_left*weights[3]+ pred_right*weights[4])
            y_pred.append(predicted_y)
        except:
            print('Some error occured.')
            y_pred.append(np.argmax(pred_whole))
        y_true.append(np.argmax(y_val[i]))
    if(matrix==True):      
        get_result_(y_pred,y_true,classes)      
    return y_true,y_pred        
          


In [95]:
def perform_preprocessing():
    train_dir='E:/BGB_example_files_test_last/images'
    if(os.path.exists('E:/BGB_example_files_test_last/train')==False):
        os.mkdir('E:/BGB_example_files_test_last/train') 
    if(os.path.exists('E:/BGB_example_files_test_last/test')==False):    
        os.mkdir('E:/BGB_example_files_test_last/test')  
    if(os.path.exists('E:/BGB_example_files_test_last/val')==False):    
        os.mkdir('E:/BGB_example_files_test_last/val')        
    regions = ['header', 'footer', 'left_body', 'right_body']
    classes=[]
    for folders in os.listdir('E:/BGB_example_files_test_last/images'):
        classes.append(folders)
    # Test directory for wholistic model
    test_dir='E:/BGB_example_files_test_last/test/whole'  
    val_dir='E:/BGB_example_files_test_last/val/whole'    
    if(os.path.exists(test_dir)== False):
        os.mkdir(test_dir)
    if(os.path.exists(val_dir)== False):
        os.mkdir(val_dir)  
    for folder in classes:
        if(os.path.exists('E:/BGB_example_files_test_last/test/whole'+folder)== False):
            os.mkdir('E:/BGB_example_files_test_last/test/whole'+folder)  
        if(os.path.exists('E:/BGB_example_files_test_last/val/whole'+folder)== False):
            os.mkdir('E:/BGB_example_files_test_last/val/whole'+folder)   
    
    # Moving files from source to test and  directory
    source_dir='E:/BGB_example_files_test_last/images'    
    dest_dir='E:/BGB_example_files_test_last/test/whole'             
    # Configure test and val data percentage here
    # we are going with train 0.8, test 0.1, val 0.1
    test_percentage=0.1
    val_percentage=0.1
    for folder in os.listdir(source_dir):
        files=os.listdir(source_dir+'/'+folder)
        count=0
        num_image_per_class=math.ceil(len(files)*test_percentage)
        shuffle(files)
        for file in files:
            if(count!=num_image_per_class):
                source=source_dir+'/'+folder+'/'+file
                dest=dest_dir+'/'+folder+'/'+file
                if(os.path.exists(dest)==False):
                    shutil.move(source,dest)
                    count+=1
    dest_dir='E:/BGB_example_files_test_last/val/whole'           
    for folder in os.listdir(source_dir):
        files=os.listdir(source_dir+'/'+folder)
        count=0
        num_image_per_class=math.ceil(len(files)*val_percentage)
        shuffle(files)
        for file in files:
            if(count!=num_image_per_class):
                source=source_dir+'/'+folder+'/'+file
                dest=dest_dir+'/'+folder+'/'+file
                if(os.path.exists(dest)==False):
                    shutil.move(source,dest)
                    count+=1 
    # Create folder structure for regional data
    for region in regions:
        if(os.path.exists('E:/BGB_example_files_test_last/train/'+region)==False):
            os.mkdir('E:/BGB_example_files_test_last/train/'+region)
        if(os.path.exists('E:/BGB_example_files_test_last/test/'+region)==False):
            os.mkdir('E:/BGB_example_files_test_last/test/'+region)
        if(os.path.exists('E:/BGB_example_files_test_last/val/'+region)==False):
            os.mkdir('E:/BGB_example_files_test_last/val/'+region)    
    for region in regions:
        for folder in classes:
            if(os.path.exists('E:/BGB_example_files_test_last/test/'+region+'/'+folder)== False):
                os.mkdir('E:/BGB_example_files_test_last/test/'+region+'/'+folder) 
            if(os.path.exists('E:/BGB_example_files_test_last/train/'+region+'/'+folder)== False):
                os.mkdir('E:/BGB_example_files_test_last/train/'+region+'/'+folder)
            if(os.path.exists('E:/BGB_example_files_test_last/val/'+region+'/'+folder)== False):
                os.mkdir('E:/BGB_example_files_test_last/val/'+region+'/'+folder)  
    if(os.path.exists('E:/BGB_example_files_test_last/train/whole')==False):            
        os.mkdir('E:/BGB_example_files_test_last/train/whole')        
    source_dir='E:/BGB_example_files_test_last/val/whole'                 
    create_regional_data(source_dir,data_type='val') 
    source_dir='E:/BGB_example_files_test_last/train/whole'          
    create_regional_data(source_dir,data_type='train')    
    source_dir='E:/BGB_example_files_test_last/test/whole'      
    create_regional_data(source_dir,data_type='test') 
    print('[Info] Preprocessing Completed')  

**CONFIGURE MODEL LEVEL GRIDSEARCH**

In [88]:
# Hyperparameters to tune
epochs_ = [5]
optimizer = ['SGD','Adam']
batch_sizes=[32,20]
learn_rate = [0.0001, 0.001]
#param_grid = dict(learn_rate=learn_rate,batch_size=batch_sizes,epochs=epochs_ )
param_grid = dict(learn_rate=learn_rate,batch_size=batch_sizes)

In [89]:
    
    def create_model(learn_rate=0.0001,optimizer='adam'):
        model =keras.applications.vgg16.VGG16(weights = None, include_top=False, input_shape = (img_width, img_height, 3))   
        for layer in model.layers:
            layer.trainable=True
        #Adding custom Layers 
        x = model.output
        x = Flatten()(x)
        x = Dropout(0.5)(x)
        x = Dense(256, activation="relu")(x)
        x = Dropout(0.5)(x)
        predictions = Dense(27, activation="softmax")(x)
        # creating the final model 
        model_ = Model( model.input,  predictions)
        # compile the model 
        optimizer=keras.optimizers.Adam(lr=learn_rate) #<--Comment out this line if optimizer is  tuned
        model_.compile(loss = "categorical_crossentropy", optimizer =optimizer, metrics=["accuracy"])  
        return model_

    def create_regional_model(learn_rate=0.0001,optimizer='adam'):
        model= create_model(learn_rate=0.0001,optimizer='adam')   
        model.load_weights('whole.hdf5')
        return model      

In [90]:
def perform_grid_search(data_generator,model_type):
    x,y=data_generator.next()
    if(model_type=='whole'):
        build_fn=create_model
    else:
        build_fn=create_regional_model    
    model = KerasClassifier(build_fn, verbose=1)
    grid_search = GridSearchCV(model, param_grid)
    print('[Info] GridSearch Started')
    grid_result=grid_search.fit(x,y)
    best_batch=grid_result.best_params_['batch_size']
    best_epoch=grid_result.best_params_['epochs'] 
    print('Best params:'+str(grid_result.best_params_))
    print('[Info] GridSearch Completed')
    return best_batch, best_epoch

In [91]:
def get_individual_models(whole_train_path='E:/BGB_example_files_test_last/images',whole_test_path='E:/BGB_example_files_test_last/val/whole' 
                          ,model_type='whole',batch_size=50,name='whole',epochs=20,enable_gridSearch=True):
    train_generator,val_generator=get_data_generator(whole_train_path,whole_test_path,img_height, img_width,500)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001)
    mcp_save = ModelCheckpoint(name+'.hdf5', save_best_only=True, monitor='acc', mode='max')
    if(model_type=='whole'):
        model=create_model()  
    else:
        model=create_regional_model()    
    print('[Info] Model Created -'+name)          
    # Wholistic Model
    print('[Info] Model Training Started -'+name)
    # Hyperparameter Configuration -- SET BEST HYPERPARAMETER OBTAINED
    if(enable_gridSearch==True):
          # RETURN the hyperparameters used in perform_grid_search
          best_batch,best_epoch=perform_grid_search(val_generator,model_type)
          perform_grid_search(val_generator,model_type)
          batch_size=best_batch
          epochs=best_epoch
          print('[Info] Train Started using best hyperparameter settings')      
    history=model.fit_generator(
    train_generator,
    epochs=epochs,steps_per_epoch = 5000/batch_size,    
    validation_data=val_generator,
    validation_steps=math.ceil(500//(batch_size)),
    callbacks=[reduce_lr,mcp_save])
    model.save(name+'.hdf5')   
    # Eval model on test data
    get_result(model,val_generator)
    return model
    

In [20]:
# Training Configurations
img_width=256
img_height=256
batch_size=10
epochs = 10
num_classes=27
whole_train_path='E:/BGB_example_files_test_last/images'
whole_test_path='E:/BGB_example_files_test_last/val/whole' 
footer_train_path='E:/BGB_example_files_test_last/train/footer'   
footer_test_path='E:/BGB_example_files_test_last/val/footer'
header_train_path='E:/BGB_example_files_test_last/train/header'       
header_test_path='E:/BGB_example_files_test_last/val/header' 
left_train_path='E:/BGB_example_files_test_last/train/left_body'
left_test_path='E:/BGB_example_files_test_last/val/left_body' 
right_train_path='E:/BGB_example_files_test_last/train/right_body'  
right_test_path='E:/BGB_example_files_test_last/val/right_body'     
classes=[]
for folders in os.listdir(whole_train_path):
    classes.append(folders)

**Execute Preprocessing**

In [94]:
perform_preprocessing()

FileNotFoundError: [Errno 2] No such file or directory: 'E:/BGB_example_files_test_last/test/whole/Collection_Note/SKM_C364e19052113410-16.png'

**Execute GridSearch on Single Model**

In [31]:
param_grid = dict(learn_rate=learn_rate,batch_size=batch_size)
train_generator,val_generator=get_data_generator(whole_train_path,whole_test_path,img_height, img_width,batch_size=200)
perform_grid_search(val_generator,model_type='whole')

Found 3686 images belonging to 27 classes.
Found 0 images belonging to 0 classes.


ValueError: Parameter grid for parameter (batch_size) needs to be a list or numpy array, but got (<class 'int'>). Single values need to be wrapped in a list with one element.

**Anchor Method**

In [ ]:
def final(img_width=256,img_height=256,batch_size=32,epochs = 10,num_classes=27,
          samples_per_epoch =5000,skip_train=False,enable_gridSearch=False,tune_stack=False,stack_weights=None):
    """
    img_width: width of images,256
    img_height: height of images, 256
    batch_size: size of trainning batch, 32
    epochs: number of epochs, 30
    num_classes: No of unique labels, 27
    samples_per_epoch: samples for training in each epoch, 1000
    skip_train: flag to skip training if trained models are already present, False
    """
    print('[Info] Pipeline Processing Started')
    # Skip this line if already preprocessed
    perform_preprocessing()
    if(skip_train==False):
        model_final_holistic=get_individual_models(whole_train_path,whole_test_path,model_type='whole',
                                              batch_size=batch_size,name='whole',epochs=epochs,enable_gridSearch=False)
        model_final_top=get_individual_models(header_train_path,header_test_path,model_type='reg',
                                              batch_size=batch_size,name='header',epochs=epochs,enable_gridSearch=False)
        model_final_bottom=get_individual_models(footer_train_path,footer_test_path,model_type='reg',
                                              batch_size=batch_size,name='footer',epochs=epochs,enable_gridSearch=False)
        model_final_left=get_individual_models(left_train_path,left_test_path,model_type='reg',
                                              batch_size=batch_size,name='left',epochs=epochs,enable_gridSearch=True)
        model_final_right= get_individual_models(right_train_path,right_test_path,model_type='reg',
                                              batch_size=batch_size,name='right',epochs=epochs,enable_gridSearch=False)
    else:
        model=create_model()
        model_final_holistic=model.load_weights('whole.hdf5')
        model_final_top=model.load_weights('header.hdf5')
        model_final_bottom=model.load_weights('footer.hdf5')
        model_final_left=model.load_weights('left.hdf5')
        model_final_right=model.load_weights('right.hdf5')
        model_final_left=get_individual_models(left_train_path,left_test_path,model_type='reg',
                                              batch_size=batch_size,name='left',epochs=epochs,enable_gridSearch=True)        
    x_val,y_val=generate_test_data('E:/BGB_example_files_test_last/test/whole',500,img_height, img_width)              #change
    x_val_top,y_val_top=generate_test_data('E:/BGB_example_files_test_last/test/header',500,img_height, img_width)     #change
    x_val_bottom,y_val_bottom=generate_test_data('E:/BGB_example_files_test_last/test/footer',500,img_height, img_width) #chnge
    x_val_left,y_val_left=generate_test_data('E:/BGB_example_files_test_last/test/left_body',500,img_height, img_width)  #Change
    x_val_right,y_val_right=generate_test_data('E:/BGB_example_files_test_last/test/right_body',500,img_height, img_width) #change
    # Default Weights
    weights=[0.4,0.2,0.2,0.1,0.1]
    if(tune_stack==True):
        stack_accuracy=[]  
        for weights in stack_weights:
            y_true,y_pred=eval_stacked_model(weights,x_val,x_val_top,x_val_bottom,x_val_left,x_val_right,y_val,
            model_final_holistic,model_final_top,model_final_bottom,model_final_left,model_final_right,classes)
            accuracy=np.round(accuracy_score(y_pred,y_true),2)
            print('---------------------')
            print('Weight:',str(weights))
            print('Accuracy:',accuracy)
            print('---------------------')
            stack_accuracy.append(accuracy)
        weights= stack_weights[np.argmax(stack_accuracy)] 
        print('Best Weights:',str(weights))       
    y_true,y_pred=eval_stacked_model(weights,x_val,x_val_top,x_val_bottom,x_val_left,x_val_right,y_val,
    model_final_holistic,model_final_top,model_final_bottom,model_final_left,model_final_right,classes,matrix=True)
    print('[Info] Model Eval Completed.')
    print(classification_report(y_true,y_pred))
    print('[Info]True Label vs Prediction Eval')
    index=[]
    for i in range(20):
        index.append(randint(0,len(y_pred)))
    for i in index:
        print('True Class:'+str(y_true[i])+' Predicted Class:'+str(y_pred[i]))
      
